# Load & Imputation

In [8]:
"""
Load dataset
"""
import pandas as pd
import numpy as np
path_to_file = "~/Desktop/new.KDD.train.csv"
data = pd.read_csv(path_to_file)
data = data.drop('Unnamed: 0',axis=1)
data.head(2)

,duration,aol,auth,bgp,courier,csnet_ns,ctf,daytime,discard,domain,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome.response
0,0,0,0,0,0,0,0,0,0,0,...,25,0.17,0.03,0.17,0.0,0.0,0.0,0.05,0.0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.0,0.0,0.0,0.00,0.0,0


# Logistic Regression in Spark

In [11]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
import pandas as pd
import numpy as np

In [12]:
#Logistic regression in Spark, with CV

#Create Spark DF
input_index = map(str, range(122))
colnames = input_index + ['label']
df = sqlContext.createDataFrame(data, colnames)   

#Change label to 'factor'
stringIndexer = StringIndexer(inputCol="label", outputCol="indexed")
si_model = stringIndexer.fit(df)
df = si_model.transform(df)

#Prep train and test Spark DF
(train_data, test_data) = df.randomSplit([0.7, 0.3])
assembler = VectorAssembler(inputCols=input_index, outputCol='features')
train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

In [ ]:
#Cross-Validation
lr = LogisticRegression(labelCol="indexed", maxIter=100, fitIntercept=True, \
                        threshold=0.5, standardization=True)
grid = ParamGridBuilder()\
    .addGrid(lr.regParam, np.logspace(-5, 5, 50))\
    .addGrid(lr.elasticNetParam, np.linspace(0,1,50))\
    .build()
evaluator = BinaryClassificationEvaluator(labelCol="indexed", metricName="areaUnderROC")
cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator)
cvModel = cv.fit(train_data)

#AUC
print "Train AUC:", evaluator.evaluate(cvModel.transform(train_data))
print "Test AUC:", evaluator.evaluate(cvModel.transform(test_data))